# Task 3 - A Simple Quantum Compiler


In [11]:
import numpy as np
import math
import functools

# Task Description:

Input assumptions:
A circuit conisisting of gates to be simplified, this is represented by a list of lists of gates. The possible gates are  H,X,Y,Z,I,RX,RY,RZ,CX,CZ
Output assumptions:
A circuit consisting of only CZ,RX,RZ

Description of most naive approach:
The most naive approach would be to simply take every single gate in the original circuit and replace them with their basic gate decomposition.

Description of main ideas for this construction:
-a palindromic circuit is a set of gates of even length where the first half evaluates to the inverse of the second half and thus produces the identity matrix if the matrices are multiplied
-Palindromic circuits of even length may be removed - it is important to note here that RX(theta),RZ(theta),RY(theta) are not self inverse, but (-theta) or (2pi-theta) of the same gate is. Also, to include multi qubit gates in a palindrome, the gates between them on all qubits they act on must be palindromic.
-the naive assumption that removing the largest palindromes from the circuit greedily, the new length is close to the minimal length of the circuit that can remain after removing any set of palindromes 
-All identity gates can be disregarded
-consecutive RX gates may be turned into a single rx gate, and the same is true for rz gates


The final function to call on a circuit to compile will be Circuit.generateandreturn_compiledcircuit()

# Creating a gate representation

- This design of gates is working off of the assumption that all angles of rotation are limited between -pi and pi, thus any input angles to paremetric gates if out of this range will give an error of out of bounds

- The superclass gate is the parent to both singlequbitgate and twoqubitgate from which all of the gates that match that are descended

-A circuit is expected here to be a list of lists, a list of lists of gates for each qubit index. So a circuit of n qubits is a list of length n.

-This assumes that the two qubit gates are given an indication in both the lists of gates for the qubits that they act on

-gates need the attributes: matrix, id (for checking type), angles (x,y,z), basisrepresentation (list of gates that represent it in the desired basis)
-gates need the functions: init (setting any angle if is paremetric), is_palindromic(g) which checks if a gate is its inverse, get_angles(), get_id(), get_basisrepresentation() (getting the gate in terms of rx, rz and cz)

-This implementation uses only the circuit decompositions that are used in qiskit

In [269]:
#Here we define the classes for the gates:
#Note: getmatrix is for testing purposes
class gate():
    def getmatrix(self):
        return self.matrix
    
    
    def is_palindromic(self,g):
        if self.id == g.id:
            return True
        
    def get_id(self):
        return self.id
    
        
    def get_basisrepresentation(self):
        return self.basisrepresentation
    
        

        

In [475]:
class singlequbitgate(gate):
    def get_xyzrotationangles(self):
        return self.angles
    
    def is_palindromic(self,g):
        if super().is_palindromic(g):
            return True
        #None of H,X,Y,Z can be represented as just one RX or RZ gate, so we define the angle comparison for equality in only RX and RZ
        #We also define this in RY so that they can be removed during remove palindromes
    
class RX(singlequbitgate):
    def __init__(self,theta):
        if theta > math.pi or theta < -math.pi:
            raise Exception() #check that the angle given is in the allowed range
        self.matrix = np.array([[np.cos(theta/2),-(np.sin(theta/2))*1j],[-(np.sin(theta/2))*1j,np.cos(theta/2)]])
        self.angles = (theta,0,0)
        self.id = ('RX')
        if theta == 0: #this means that it is the identity
            self.basisrepresentation = []
        else:
            self.basisrepresentation = [self]
    
    #For parametric gates, if theta for the comparison gate = -theta for this gate, then they are inverse and thus palindromic
    def is_palindromic(self,g):
        if g.get_id()[0] == 'C':
            return False
        if g.get_xyzrotationangles()[0] == -self.angles[0] and g.get_xyzrotationangles()[1] == 0 and g.get_xyzrotationangles()[2] == 0:
            return True
        
        return False


class RZ(singlequbitgate):
    def __init__(self,theta):
        if theta > math.pi or theta < -math.pi:
            raise Exception()
        self.matrix = np.array([[np.cos(theta/2)-(np.sin(theta/2))*1j,0],[0,np.cos(theta/2)+(np.sin(theta/2))*1j]])
        self.angles = (0,0,theta)
        self.id = ('RZ')
        if theta == 0:
            self.basisrepresentation = []
        else:
            self.basisrepresentation = [self]
        
        
    def is_palindromic(self,g):
        if g.get_id()[0] == 'C':
            return False
        if g.get_xyzrotationangles()[2] == -self.angles[2] and g.get_xyzrotationangles()[0] == 0 and g.get_xyzrotationangles()[1] == 0:
            return True
        
        return False
    
class RY(singlequbitgate):
    
    def __init__(self,theta):
        if theta > math.pi or theta < -math.pi:
            raise Exception()
        self.matrix = np.array([[np.cos(theta/2)-(np.sin(theta/2))*1j,0],[0,np.cos(theta/2)+(np.sin(theta/2))*1j]])
        self.angles = (0,theta,0)
        self.id = ('RY')
        self.basisrepresentation = [RX(-math.pi/2),RZ(theta),RX(math.pi/2)]
    
    def is_palindromic(self,g):
        if g.get_id()[0] == 'C':
            return False
        if g.get_xyzrotationangles()[1] == -self.angles[1] and g.get_xyzrotationangles()[0] == 0 and g.get_xyzrotationangles()[2] == 0:
            return True
        
class H(singlequbitgate):
    def __init__(self):
        self.angles = (math.pi/2,0,math.pi/2)
        self.id = ('H')
        self.basisrepresentation = [RZ(math.pi/2),RX(math.pi/2),RZ(math.pi/2)]
        
class X(singlequbitgate):
    def __init__(self):
        self.angles = (math.pi,0,math.pi)
        self.id = ('X')
        self.basisrepresentation = [RX(-math.pi/2),RZ(math.pi),RX(math.pi/2),RZ(math.pi)]
        
    
class Y(singlequbitgate):
    def __init__(self):
        self.angles = (math.pi,math.pi/2,math.pi/2)
        self.id = ('Y')
        self.basisrepresentation = [RZ(math.pi/2),RX(-math.pi/2),RZ(math.pi),RX(math.pi/2),RZ(math.pi/2)]
        

class Z(singlequbitgate):
    def __init__(self):
        self.angles = (0,0,math.pi)
        self.id = ('Z')
        self.basisrepresentation = [RZ(math.pi)]

    

class I(singlequbitgate):
    def __init__(self):
        self.matrix = np.array([[1,0],[0,1]])
        self.angles = (0,0,0)
        self.id = 'I'
        self.basisrepresentation = []
        
    def is_palindromic(self,g):
        if g.get_xyzrotationangles() == (0,0,0):
            return True
        return super().is_palindromic(g)
    
    


# Description of palindrome finding protocol

The process of moving palindromes shall be called remove_palindromes(self).

The steps of this function are defined as such:
-for each qubit in the circuit, call removepalindromesfromqubit(index)

removepalindromesfromqubit is defined as:
-while removelargestpalindrome() > 1: #this is when there is no palindrome left
-call removelargestevenpalindrome()

removelargestevenpalindrome is defined as:
-find start and end indices of largest palindrome in qubit line
-set that line of gates to the original list wohtout the palindrome gates


When comparing two controlled gates the code must compare the gates between both pairs of qubits the two act on.



In [420]:
#defining the two qubit gates

class twoqubitgate(gate):
    def __init__(self,controlindex,actindex):
        self.controlindex = controlindex
        self.actindex = actindex
        
    def get_actindex(self):
        return self.actindex
        
    def get_controlindex(self):
        return self.controlindex
    
    def is_palindromic(self,g,qubitindexon,circuit):
        if g.id == self.id and g.get_controlindex() == self.controlindex and g.get_actindex == self.actindex:
            if qubitindexon > min(self.actindex,self.controlindex): #if the index passed to it is larger than one of the qubits the gate is acting on, then that means that this has already been called on these gates, and the fact that they have not been removed means that they were not part of a palindrome before, so will still not be now.
                return False
            
            lengthbetweengates = circuit[max(self.actindex,self.controlindex)].index(g) - circuit[max(self.actindex,self.controlindex)].index(self)
            
            #if the length of the gates between the two gates is odd, it cannot be a palindrome
            if (lengthbetweengates )%2 == 1:
                return False
            
            #if the length is even, check if it is a palindrome
            for i in range((lengthbetweengates)/2):
                if circuit[qubitindexon][i].equals(circuit[qubitindexon][lengthbetweengates-i]) != True:
                    return False
            
            return True
            
        return False
    
class CX(twoqubitgate):
    def __init__(self,controlindex,actindex):
        if controlindex == actindex:
            raise Exception
        self.controlindex = controlindex
        self.actindex = actindex
        self.id = ('CX')
        #if self.actindex > self.controlindex:
        #    self.matrix = np.array([[0.+0.j 0.+0.j 1.+0.j 0.+0.j],[0.+0.j 1.+0.j 0.+0.j 0.+0.j],[1.+0.j 0.+0.j 0.+0.j 0.+0.j],[0.+0.j 0.+0.j 0.+0.j 1.+0.j]])
        #else:
        #    self.matrix = np.array([[1.+0.j 0.+0.j 0.+0.j 0.+0.j],[0.+0.j 0.+0.j 0.+0.j 1.+0.j],[0.+0.j 0.+0.j 1.+0.j 0.+0.j],[0.+0.j 1.+0.j 0.+0.j 0.+0.j]])
            
        self.basisrepresentation = [H(),CZ(controlindex,actindex),H()]

class CZ(twoqubitgate):
    def __init__(self,controlindex,actindex):
        if controlindex == actindex:
            raise Exception
        self.controlindex = controlindex
        self.actindex = actindex
        self.id = ('CZ')
        #if self.actindex > self.controlindex:
        #    self.matrix = np.array([[1.+0.j 0.+0.j 0.+0.j 0.+0.j],[0.+0.j 1.+0.j 0.+0.j 0.+0.j],[0.+0.j 0.+0.j 0.+0.j 1.+0.j],[0.+0.j 0.+0.j 1.+0.j 0.+0.j]])
        #else:
        #    self.matrix = np.array([[1.+0.j 0.+0.j 0.+0.j 0.+0.j],[0.+0.j 0.+0.j 0.+0.j 1.+0.j],[0.+0.j 0.+0.j 1.+0.j 0.+0.j],[0.+0.j 1.+0.j 0.+0.j 0.+0.j]])
            
        self.basisrepresentation = [self]

# Circuit design

Circuits shall be represented as a list of lists of gates.

The compilation routine will first remove palindromes, then change the gates to basis gates, then flatten runs of gates of the same type, then remove any remaining palindromes.

Whilst the last two steps could be run continually until neither does anything, this could lead to very poor runtime complexity. As the runtime is already numberofgatesononequbit^3 or ^4 depending on input, this has been avoided here for ease of use.

In [476]:
class Circuit():
    def __init__(self,numqubits):
        self.circuit = []
        for i in range(numqubits):
            self.circuit.append([])
        self.reducedcircuit = []
        
    def add_gate(self,qubitindex,g):
        self.circuit[qubitindex].append(g)
        #If a controlled gate, put reference in both qubits
        if g.get_id() == 'CX' or g.get_id() == 'CZ':
            if qubitindex == g.get_controlindex():
                self.circuit[g.get_actindex()].append(g)
            else:
                self.circuit[g.get_controlindex()].append(g)
                
    def get_initcirc(self):
        return self.circuit
     
    #function to combine two parametric gates of the same type
    def combinetwoparametrigates(self,a,b):
        index = 0
        if b.get_id() == 'RZ':
            index == 2
        newtheta = (((a.get_xyzrotationangles()[index]+b.get_xyzrotationangles()[index])+(math.pi))% (2*math.pi))-math.pi
        if b.get_id() == 'RZ':
            return RZ(newtheta)
        else:
            return RX(newtheta)
        
    def generateandreturn_compiledcircuit(self):
        self.reducedcircuit = self.circuit
        #first remove any existing palindromes so that a smaller number of gates remain to replace with their representations in the basis gates
        self.removepalindromes()
        
        #This step replaces all of the gates with their basis representations, if they are the I gate or a parametric gate with all 0 angles, it simply appends the empty list thus deleting it
        newreducedcircuit = []
        for index in  range(len(self.reducedcircuit)):
            newreducedcircuit.append([])
            for g in self.reducedcircuit[index]:
                newreducedcircuit[index].extend(g.get_basisrepresentation())
        self.reducedcircuit = newreducedcircuit
        
        
        flattenparametricgates  = lambda circ: functools.reduce(self.combinetwoparametrigates, circ, RX(0))
        #go through the qubits and turn all runs of RX gates, or runs of RZ gates into a single RX or RZ gate
        for index in range(len(self.reducedcircuit)):
            newrow = []
            glen = len(self.reducedcircuit[index])
            gindex = 0
            startindex = 0
            
            if glen <= 1:
                newrow = self.reducedcircuit[index]
                break
            
            while gindex < glen-2:
                nextissametype = True
                startindex = gindex
                while nextissametype:
                    gindex+=1
                    if self.reducedcircuit[index][startindex].get_id()[0] == 'C':
                        nextissametype = False
                    if self.reducedcircuit[index][startindex].get_id() != self.reducedcircuit[index][gindex].get_id():
                        nextissametype = False
                if self.reducedcircuit[index][startindex].get_id()[0] != 'C':
                    newrow.append(flattenparametricgates(self.reducedcircuit[index][startindex:gindex]))
                else:
                    newrow.extend(self.reducedcircuit[index][startindex:gindex])
            self.reducedcircuit[index] = newrow
                
                
        #finally remove any palindromes that now remain just in the hopes of shrinking it a little more
        self.removepalindromes()
        
        return self.reducedcircuit
            
    #This function iterates through each qubit and calls removepalindromesfromqubit on each to get rid of all palindromes from the qubit's gates
    def removepalindromes(self):
        for index in range(len(self.circuit)):
            self.removepalindromesfromqubit(index)
            
    
    #this function calls removelargestpalindromefromqunbitsgates until it returns 1 which indicates that there are no more even palindromic circuits in this qubit's gates     
    def removepalindromesfromqubit(self,index):
        lengthpalindrome = len(self.circuit[index])
        while(lengthpalindrome > 1):
            lengthpalindrome = self.removelargestpalindromefromqubitsgates(index)
    
    
    #This function iterates looks for the middle index and length of the largest even palindrome in the gate list, if there is one, it returns a copy of the qubit's without those in the first found largest palindrome found
    def removelargestpalindromefromqubitsgates(self,index):
        midindexpalindrome = -1
        length = 0
        maxpalindrome = 0
        qubitgates = self.reducedcircuit[index]
        for i in range(1,len(qubitgates)): #iteratively set each index as the middle of the palindrome
            for j in range(min(i,len(qubitgates)-i)): #check up to the nearest end how long a palindrome can be made with this gate as the center               
                if qubitgates[i+j].get_id()[0] != 'C': #requires different params if is controlled gate
                    print(qubitgates[i+j].get_id()[0])
                    if qubitgates[i+j].is_palindromic(qubitgates[i-j-1]): #if the gates are inverses, increase the length of the possible palindrome by 2
                        length += 2
                    else: #if they aren't equal you have reached the end of the palindrome
                        break
                else:
                    if qubitgates[i+j].is_palindromic(qubitgates[i-j-1],index,self.reducedcircuit): #if the gates are inverses, increase the length of the possible palindrome by 2
                        length += 2
                    else: #if they aren't equal you have reached the end of the palindrome
                        break
                    
            if length > maxpalindrome: #if the latest palindrome found is the longest, set the found length and start point to those from this start point
                midindexpalindrome = i
                maxpalindrome = length
                length = 1
        if maxpalindrome > 1: #if there is an even palindrome, take the gates from outside it and set those to the new list of gates for this qubit, if not return 1 to indicate no even palindrome
            gateswithoutpalindrome = qubitgates[:midindexpalindrome-(maxpalindrome//2)]
            gateswithoutpalindrome.extend(qubitgates[midindexpalindrome+((maxpalindrome//2)):])
            
        else:
            return 1

        self.reducedcircuit[index] = gateswithoutpalindrome
        return maxpalindrome
    
        #Note: we don't do anything different if there is a controlled gate because the parts with the palindrome with alrger qubit index are checked later, this is at slight cost to efficiency
    
    
            
        
            
            

# Tests:

In [477]:
#Test that all the gates get turned into their basis gates:

c = Circuit(11)
c.add_gate(0,I())
c.add_gate(1,X())
c.add_gate(2,Y())
c.add_gate(3,Z())
c.add_gate(4,RX(0.2))
c.add_gate(5,RY(0.3))
c.add_gate(6,RZ(0.1))
c.add_gate(7,CX(7,8))
c.add_gate(9,CZ(9,10))

print(c.get_initcirc())

print(c.generateandreturn_compiledcircuit())



[[<__main__.I object at 0x7fd057153df0>], [<__main__.X object at 0x7fd057153250>], [<__main__.Y object at 0x7fd0571534c0>], [<__main__.Z object at 0x7fd05714dc10>], [<__main__.RX object at 0x7fd05714b9d0>], [<__main__.RY object at 0x7fd05714b4f0>], [<__main__.RZ object at 0x7fd056b3bac0>], [<__main__.CX object at 0x7fd056b3b670>], [<__main__.CX object at 0x7fd056b3b670>], [<__main__.CZ object at 0x7fd056b3b6a0>], [<__main__.CZ object at 0x7fd056b3b6a0>]]
[[], [<__main__.RX object at 0x7fd057153b20>, <__main__.RZ object at 0x7fd05714de20>, <__main__.RX object at 0x7fd05714d3d0>, <__main__.RZ object at 0x7fd05714df70>], [<__main__.RZ object at 0x7fd05714b0d0>, <__main__.RX object at 0x7fd05714b760>, <__main__.RZ object at 0x7fd05714b4c0>, <__main__.RX object at 0x7fd05714bac0>, <__main__.RZ object at 0x7fd05714b040>], [<__main__.RZ object at 0x7fd05714bf70>], [<__main__.RX object at 0x7fd05714b9d0>], [<__main__.RX object at 0x7fd056b3b7c0>, <__main__.RZ object at 0x7fd056b3baf0>, <__main

In [478]:
#Test that it deletes palindromes:
c = Circuit(1)
c.add_gate(0,X())
c.add_gate(0,Y())
c.add_gate(0,Y())
c.add_gate(0,X())

print(c.get_initcirc())

print(c.generateandreturn_compiledcircuit())

a = [0,1,2,3]


[[<__main__.X object at 0x7fd05714b9a0>, <__main__.Y object at 0x7fd05714dc10>, <__main__.Y object at 0x7fd05714eeb0>, <__main__.X object at 0x7fd056d05b20>]]
Y
Y
X
X
[[]]


In [479]:
#Test that it deletes palindromes:
c = Circuit(1)
c.add_gate(0,Z())
c.add_gate(0,X())
c.add_gate(0,Y())
c.add_gate(0,Y())
c.add_gate(0,X())
c.add_gate(0,Y())

print(c.get_initcirc())

print(c.generateandreturn_compiledcircuit())



[[<__main__.Z object at 0x7fd0571c9e20>, <__main__.X object at 0x7fd05714e3d0>, <__main__.Y object at 0x7fd0571c95e0>, <__main__.Y object at 0x7fd0569f10d0>, <__main__.X object at 0x7fd057153ee0>, <__main__.Y object at 0x7fd056bfc700>]]
X
Y
Y
X
Y
X
Y
Y
R
R
[[<__main__.RZ object at 0x7fd0571534f0>, <__main__.RX object at 0x7fd057153e20>, <__main__.RZ object at 0x7fd056b6bc10>]]


In [480]:
#Test that it deletes palindromes:
c = Circuit(2)
c.add_gate(0,CZ(0,1))
c.add_gate(0,X())
c.add_gate(0,Y())
c.add_gate(0,Y())
c.add_gate(0,X())
c.add_gate(0,CZ(0,1))

print(c.get_initcirc())

print(c.generateandreturn_compiledcircuit())



[[<__main__.CZ object at 0x7fd056a1e3a0>, <__main__.X object at 0x7fd0571534f0>, <__main__.Y object at 0x7fd0571c98b0>, <__main__.Y object at 0x7fd056c96dc0>, <__main__.X object at 0x7fd056b6b9d0>, <__main__.CZ object at 0x7fd056b6baf0>], [<__main__.CZ object at 0x7fd056a1e3a0>, <__main__.CZ object at 0x7fd056b6baf0>]]
X
Y
Y
X
X
[[], []]


In [481]:
#Test that it deletes palindromes:
c = Circuit(2)
c.add_gate(0,CZ(0,1))
c.add_gate(0,X())
c.add_gate(0,Y())
c.add_gate(0,Y())
c.add_gate(0,X())
c.add_gate(1,Y())
c.add_gate(0,CZ(0,1))

print(c.get_initcirc())

print(c.generateandreturn_compiledcircuit())



[[<__main__.CZ object at 0x7fd057157910>, <__main__.X object at 0x7fd056a1e850>, <__main__.Y object at 0x7fd0569f7670>, <__main__.Y object at 0x7fd056b6be20>, <__main__.X object at 0x7fd05714eaf0>, <__main__.CZ object at 0x7fd0569d41c0>], [<__main__.CZ object at 0x7fd057157910>, <__main__.Y object at 0x7fd056b3b4c0>, <__main__.CZ object at 0x7fd0569d41c0>]]
X
Y
Y
X
X
Y
R
R
R
R
[[], [<__main__.CZ object at 0x7fd057157910>, <__main__.RZ object at 0x7fd056b6b4f0>, <__main__.RX object at 0x7fd056b6bd60>, <__main__.RZ object at 0x7fd056b6b220>, <__main__.RX object at 0x7fd056b6bf40>]]


# Room for improvement:

-This is mainly written in Jupyter Notebook for demonstrative purposes, there are also multiple areas where with more time one could greatly speed up the efficiency.
-There are other techniques for compilation involving seaching for patterns that are deletable such as certain sets of cnots and hadamards
-Some proof on using palindromes to minimise circuits, and minimising sets of rx and rz perhaps looking at their corresponding u3 gates could also be done